In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Notebook 03 — Variance Reduction in Monte Carlo Simulation

Goal:
- Reduce estimator variance in Monte Carlo pricing
- Implement:
  1. Antithetic variates
  2. Control variates
- Quantify variance reduction numerically

This notebook extends the Monte Carlo engine with
industry-standard variance reduction techniques
used in large-scale risk and pricing systems.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

In [3]:
# Market parameters
S0 = 100.0
K = 100.0
T = 1.0
r = 0.05
mu = r
sigma = 0.2

# Monte Carlo
M = 100_000

In [4]:
def mc_european_call_price(S0, K, T, r, mu, sigma, M):
    Z = np.random.normal(0, 1, size=M)
    ST = S0 * np.exp((mu - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    payoff = np.maximum(ST - K, 0)
    price = np.exp(-r * T) * payoff.mean()
    return price, payoff

In [5]:
price_mc, payoff_mc = mc_european_call_price(
    S0, K, T, r, mu, sigma, M
)

price_mc

np.float64(10.473891960702577)

In [6]:
def mc_antithetic_call_price(S0, K, T, r, mu, sigma, M):
    Z = np.random.normal(0, 1, size=M // 2)
    Z_antithetic = -Z

    Z_all = np.concatenate([Z, Z_antithetic])

    ST = S0 * np.exp((mu - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z_all)
    payoff = np.maximum(ST - K, 0)

    price = np.exp(-r * T) * payoff.mean()
    return price, payoff

In [7]:
price_anti, payoff_anti = mc_antithetic_call_price(
    S0, K, T, r, mu, sigma, M
)

price_anti

np.float64(10.448405516328588)

In [8]:
np.var(payoff_mc), np.var(payoff_anti)

(np.float64(239.88196813084872), np.float64(238.9658958586914))

In [9]:
def mc_control_variate_call_price(S0, K, T, r, mu, sigma, M):
    Z = np.random.normal(0, 1, size=M)
    ST = S0 * np.exp((mu - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    payoff = np.maximum(ST - K, 0)
    control = ST

    expected_control = S0 * np.exp(mu * T)

    cov = np.cov(payoff, control)
    beta = cov[0, 1] / cov[1, 1]

    adjusted_payoff = payoff + beta * (expected_control - control)

    price = np.exp(-r * T) * adjusted_payoff.mean()
    return price, adjusted_payoff

In [10]:
price_cv, payoff_cv = mc_control_variate_call_price(
    S0, K, T, r, mu, sigma, M
)

price_cv

np.float64(10.445839428999781)

In [11]:
np.var(payoff_mc), np.var(payoff_cv)

(np.float64(239.88196813084872), np.float64(34.71141234959297))

In [12]:
import pandas as pd

summary = pd.DataFrame({
    "Method": ["Vanilla MC", "Antithetic", "Control Variate"],
    "Price": [price_mc, price_anti, price_cv],
    "Payoff Variance": [
        np.var(payoff_mc),
        np.var(payoff_anti),
        np.var(payoff_cv)
    ]
})

summary

,Method,Price,Payoff Variance
0,Vanilla MC,10.473892,239.881968
1,Antithetic,10.448406,238.965896
2,Control Variate,10.445839,34.711412
